In [42]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # setting ignore as a parameter and further adding category
from whispr import *
import os


In [85]:
# gene expression. 2.5ul rxns, 75% txtl
rxn_vol = 10
source_plate_type = '384PP_AQ_BP' 

os.getcwd()
folder = os.getcwd() + '/Experiments/250117_KO/'
sp_plasmids_file = folder + '250117_plasmids_amounts.xlsx'
sp_plasmids = pd.read_excel(sp_plasmids_file, index_col = 0, engine='openpyxl')
sp_plasmids = sp_plasmids[~sp_plasmids['Well'].isna()]

layout_genex_file = folder + '250117_plate_map.csv'
#layout_genex = pd.xlsx(layout_genex_file, index_col = 0, dtype = str)
layout_genex = pd.read_csv(layout_genex_file, index_col = 0, dtype = str)

mt_genex_file = folder + '250117_table_amounts.csv'
# mt_genex = pd.read_excel(mt_genex_file, index_col = 0, engine='openpyxl').fillna(0)
#mt_genex = pd.read_excel(mt_genex_file, index_col = 0, dtype = str).fillna(0)
mt_genex = pd.read_csv(mt_genex_file, index_col = 0, dtype = str).fillna(0)

checkInputs(sp_plasmids, mt_genex,source_plate_type)

vol_table_df = generateVolumeTable(mt_genex, sp_plasmids, rxn_vol = 0.25*rxn_vol, total_vol = rxn_vol, fill_with='Water', multiRpW = False)

#specify rxn_vol (default = 2.5) and total_vol (default = 10) if you'd like to change the volume of each individual replicate or the total reaction volume
protocol_genex_df = writeProtocol(source_plate_type, vol_table_df, layout_genex,sp_plasmids, update_source_vol=folder+'plasmids_sp_updated.xlsx')


A1 chi6
A2 chi6
A3 chi6
A4 chi6
A5 chi6
A6 chi6
A7 chi6
A8 chi6
A9 chi6
A10 chi6
A11 chi6
A12 chi6
A13 chi6
A14 chi6
A15 chi6
A16 chi6
A17 chi6
A18 chi6
A19 chi6
A20 chi6
A21 chi6
A22 chi6
A23 chi6
A24 chi6
A25 chi6
A26 chi6
A27 chi6
A28 chi6
A29 chi6
A30 chi6
A31 chi6
A32 chi6
A33 chi6
A34 chi6
Volumes used from each well for this protocol:
{'1_B16': 40.425, '1_B17': 8.475, '1_C1': 5.95, '1_C12': 6.8, '1_C3': 1.225, '1_C4': 7.5, '1_C5': 0.7, '1_C6': 1.25, '1_C13': 7.9, '1_C8': 0.625, '1_C9': 0.475, '1_C10': 1.575, '1_C11': 2.1}


In [86]:
protocol_genex_df = protocol_genex_df[0]

In [87]:
# everything gets txtl

# get destination wells
for layout_id, layout in enumerate([layout_genex]):    
    # reads plate layout and assigns wells to each reaction (rxn_loc; dict)
    labels = pd.unique(np.concatenate(layout.values))
    rxn_loc = {}
    for l in labels:
        if type(l) is str:
            index = layout[layout.isin([l])].stack().index
            rxn_loc[l] = []
            for i in index:
                rxn_loc[l].append(str(i[0]) + str(i[1]))

#if you need more wells than one for the reaction, enter the wells where your Txtl is stored in a list format below ex. ["P7", "P8", "P9"]
txtl_well = ['P7', "P8", "P9", "P10", "P11", "P12", "P13"]
# txtl_well = "P7"
if type(txtl_well) == str:
    for label, dwell in rxn_loc.items():
        for d in dwell:
            print(label)
            txtl_vol = vol_table_df[vol_table_df['Label'] == label].iloc[0,1:].sum()/0.25*0.75*1000 # add txtl to make 75%
            row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': txtl_well,
                                        'Destination Plate Name':'Destination[1]', 'Destination Well': d, 'Transfer Volume': txtl_vol}
            protocol_genex_df = protocol_genex_df.append(row, ignore_index = True)
            #protocol_genex_df = protocol_genex_df.append(row)
elif type(txtl_well) == list:
    print("here")
    running_txtl_vol = 60000
    well_index = 0
    for label, dwell in rxn_loc.items():
        for d in dwell:
            print(label)
            txtl_vol = vol_table_df[vol_table_df['Label'] == label].iloc[0,1:].sum()/0.25*0.75*1000 # add txtl to make 75%
            running_txtl_vol = running_txtl_vol - txtl_vol
            if running_txtl_vol > 20000:
                row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': txtl_well[well_index],
                                            'Destination Plate Name':'Destination[1]', 'Destination Well': d, 'Transfer Volume': txtl_vol}
                protocol_genex_df = protocol_genex_df.append(row, ignore_index = True)
            elif running_txtl_vol < 20000:
                well_index = well_index + 1
                try:
                    row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': txtl_well[well_index],
                                                'Destination Plate Name':'Destination[1]', 'Destination Well': d, 'Transfer Volume': txtl_vol}
                    protocol_genex_df = protocol_genex_df.append(row, ignore_index = True)
                    running_txtl_vol  = 60000 - txtl_vol
                except: 
                    print("Add more TXTL wells to your list")
            #protocol_genex_df = protocol_genex_df.append(row)


protocol_genex_df.to_csv(folder+'protocol_genex.csv',index = False)

if type(txtl_well) is str:
      txtl_well = [txtl_well]

print(f"TXTL needed: {protocol_genex_df[protocol_genex_df['Source Well'].isin(txtl_well)]['Transfer Volume'].sum()/1000} ul")
print(f"Wells needed: {np.ceil(protocol_genex_df[protocol_genex_df['Source Well'].isin(txtl_well)]['Transfer Volume'].sum()/1000 / 40)}")
# carry out gene expression reaction

here
A1
A2
A3
A4
A5
A6
A7
A8
A9
A10
A11
A12
A13
A14
A15
A16
A17
A18
A19
A20
A21
A22
A23
A24
A25
A26
A27
A28
A29
A30
A31
A32
A33
A34
TXTL needed: 255.0 ul
Wells needed: 7.0


In [63]:
protocol_genex_df[protocol_genex_df['Source Well'].isin(txtl_well)]

,Source Plate Name,Source Plate Type,Source Well,Destination Plate Name,Destination Well,Transfer Volume


In [24]:
# dilution. everything gets Tris, for 20x -> 57uL

##### doing it by hand if not too many reactions #####

# protocol_dilution_df = pd.DataFrame(columns=protocol_genex_df.columns)
# source_plate_type = '6RES_AQ_B' 
# tris_well = 'A1'
# tris_vol = 57*1000

# for dwell in protocol_genex_df['Destination Well'].unique():
#     row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': tris_well,
#                                 'Destination Plate Name':'Destination[1]', 'Destination Well': dwell, 'Transfer Volume': tris_vol}
#     protocol_dilution_df = protocol_dilution_df.append(row, ignore_index = True)

# protocol_dilution_df.to_csv('protocols/221024_arpae_dilutions.csv',index = False)

In [25]:
# from source plate with buffer, mix into 96

# from source plate with genex, mix the diluted txtl

In [39]:
# biosynthesis. 25ul rxns, 2.5ul of diluted txtl

# buffer source plate
sp_buffers_file = folder + 'buffers_sp.xlsx'
sp_buffers= pd.read_excel(sp_buffers_file, index_col = 0, engine = 'openpyxl')
sp_buffers = sp_buffers[~sp_buffers['Well'].isna()]

# HEPES reservoir (actually same plate as Tris)
sp_hepes = pd.DataFrame(columns=sp_buffers.columns)
sp_hepes = sp_hepes.append({'Label':'HEPES', 'Well':'A2,A3', 'Concentration':'', 'Volume':'2000,2000'}, ignore_index = True)

# gene expression source plate (diluted w/ Tris)
sp_genex = sp_from_layout(layout_genex, 60)

# group source plates
sp_types = ['384PP_AQ_BP','6RES_AQ_BP2','384PP_AQ_BP'] # triple check this
sps = [sp_buffers,sp_hepes,sp_genex]

# get mixing table
mt_biosyn_file = folder + '040323_mixing-table-biosyn.csv'
mt_biosyn = pd.read_csv(mt_biosyn_file, index_col = 0, dtype = str).fillna(0)
mt_biosyn = mt_biosyn[~mt_biosyn.index.isna()]

# check formats
checkInputs(sps,mt_biosyn,sp_types)

# layouts for destination plate(s)
layout_biosyn1_file = folder + '040323_plate-layout-biosyn.csv' 
layout_biosyn1 = pd.read_csv(layout_biosyn1_file, index_col = 0, dtype = str)
layout_biosyn1 = layout_biosyn1[~layout_biosyn1.index.isna()]

# filename = '221018_arpae_biosyn_neg.csv'
# layout_biosyn2_file = 'plate_layouts/'+filename 
# layout_biosyn2 = pd.read_csv(layout_biosyn2_file, index_col = 0, dtype = str)
# layouts = [layout_biosyn1,layout_biosyn2]

vol_table_df = generateVolumeTable(mt_biosyn, sps, rxn_vol = 22.5, total_vol = 25, fill_with='HEPES')

protocol_biosyn_dfs = writeProtocol(sp_types, vol_table_df, layout_biosyn1,sps, update_source_vol= folder +'combined_sp_updated.xlsx')

#protocol_biosyn_df.to_csv('protocols/221018_arpae_biosyn.csv',index = False)

NameError: Need more volume of tdcb_2.5 to complete reaction ser_0. Add another well to source plate.

In [27]:
protocol_biosyn_dfs[0].to_csv(folder+'biosyn_protocol_384.csv',index = False)
protocol_biosyn_dfs[1].to_csv(folder+'biosyn_protocol_6RES.csv',index = False)